In [1]:

from gurobipy import *
import numpy as np
import math
import copy
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# ---------- Data ----------
import numpy as np
import math
from gurobipy import Model, GRB, quicksum

# Open the file and read the data
with open('data_small_multiTW2.txt', 'r') as f:
    data = f.readlines()

CVRPTW = []
for line in data:
    parts = line.split()
    loc_id = int(parts[0])
    x_coord = int(parts[1])
    y_coord = int(parts[2])
    demand = int(parts[3])
    service_time = int(parts[4])
    num_tw = int(parts[5])
    time_windows = []
    for n in range(num_tw):
        start = int(parts[6 + 2*n])
        end = int(parts[7 + 2*n])
        time_windows.append((start, end))

    # Combine location data with the time windows
    CVRPTW.append([loc_id, x_coord, y_coord, demand, service_time] + time_windows)

# Convert list to numpy array for easier manipulation
CVRPTW = np.array(CVRPTW, dtype=object)

Nodes = CVRPTW[:, 0]  # nodes
n = len(Nodes)  # number of nodes

Vehicles = {0, 1}

xcoord = CVRPTW[:, 1]  # x coordinates
ycoord = CVRPTW[:, 2]  # y coordinates
q = CVRPTW[:, 3]  # demand
service_duration = CVRPTW[:, 4]  # service time

# Extract time windows for each node
time_windows = CVRPTW[:, 5:].tolist()

Q = 130  # vehicle capacity
BIGM = 12345  # big M

# Calculate the distance matrix
distance = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        if i != j:
            distance[i, j] = math.sqrt((xcoord[i] - xcoord[j])**2 + (ycoord[i] - ycoord[j])**2)
            # if i == j:
                # distance[i, j] = BIGM  # Set a large penalty for self-loops\n",
# ---------- Optimization Model ----------

# Initialize the model
m = Model("CVRPTW_MultiTW")

# Decision Variables
# x_{ijv}: Binary variable, 1 if vehicle v travels from i to j, 0 otherwise
edge = {}
for i in Nodes:
    for j in Nodes:
        for v in Vehicles:
            edge[i, j, v] = m.addVar(vtype=GRB.BINARY, lb=0, name='x_%s_%s_%s' % (i, j, v))

# service_start{ivw}: Time at which vehicle v starts service at node i at time window w
service_start = {}
for i in Nodes:
    for v in Vehicles:
        for w in range(len(time_windows[int(i)])):  # int(i) converts numpy float to int for indexing
            service_start[i, v, w] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name='t_%s_%s_%s' % (i, v, w))

# z_{ivw}: Binary variable, 1 if node i is visited by vehicle v during time window w
z = {}
for i in Nodes:
    for v in Vehicles:
        for w in range(len(time_windows[int(i)])):
            z[i, v, w] = m.addVar(vtype=GRB.BINARY, lb=0, name='z_%s_%s_%s' % (i, v, w))

# if time window is not open
# wait_time = {}
# for i in Nodes:
#     for v in Vehicles:
#         wait_time[i, v] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name='wait_%s_%s' % (i, v))

m.update()


# Objective Function
obj = quicksum(distance[i,j]*edge[i,j,v] for i in Nodes for j in Nodes for v in Vehicles)
m.setObjective(obj)
m.ModelSense = GRB.MINIMIZE
m.update()

# Constraints

# each node is visited by exactly one vehicle 1
con0 = {}
for v in Vehicles:
    for i in Nodes:
        con0[i,i,v] = m.addConstr(
            edge[i,i,v] == 0, 'con0[' + str(i) + ',' + str(i) + ',' + str(v) + ']'
        )

# 
con1 = {}
for i in Nodes:
    for v in Vehicles:
        con1[i,j,v] = m.addConstr(
            quicksum(edge[i,j,v] for j in Nodes) == quicksum(z[i, v, w] for w in range(len(time_windows[int(i)]))), 'con1[' + str(i) + ',' + str(j) + ',' + str(v) + ']'
        )



con11 = {}
for i in Nodes:
    if i != 0:
        con11[i,v,w] = m.addConstr(
            quicksum(z[i,v,w] for v in Vehicles for w in range(len(time_windows[int(i)]))) == 1, 'con11[' + str(i) + ',' + str(v) + ',' + str(w) + ']'
        )

    
# each vehicle starts and ends at the depot
con2 = {}
con3 = {}
for v in Vehicles:
    con2[v] = m.addConstr(
        quicksum(edge[0,i,v] for i in Nodes if i != 0) == 1, 'con2[' + str(v) + ']'
    )
    con3[v] = m.addConstr(
        quicksum(edge[i,0,v] for i in Nodes if i != 0) == 1, 'con3[' + str(v) + ']'
    )


# Capacity constraint
con4 = {}
for v in Vehicles:
    con4[v] = m.addConstr(
        quicksum(q[i]*z[i,v,w] for i in Nodes for w in range(len(time_windows[int(i)]))) <= Q, 'con4[' + str(v) + ']'
    )


# Flow conservation excluding self-loops
con5 = {}
for i in Nodes:
    for v in Vehicles:
        con5[i, j, v] = m.addConstr(
            quicksum(edge[i, j, v] for j in Nodes) == quicksum(edge[j, i, v] for j in Nodes),
            'con5[' + str(i) + ',' + str(j) + ',' + str(v) + ']'
        )


# time window constraints
con6 = {}
con7 = {}
con8 = {}
con9 = {}
for i in Nodes:
    for v in Vehicles:
        for w in range(len(time_windows[int(i)])):
            a_iw, b_iw = time_windows[int(i)][w]
            print(a_iw)
            print(b_iw)
            con6[i,v,w] = m.addConstr(
                service_start[i, v, w] >= a_iw , 'con6[' + str(i) + ',' + str(v) + ',' + str(w) + ']'
                )
            con7[i,v,w] = m.addConstr(
                service_start[i, v, w] + service_duration[int(i)]  <= b_iw, 'con7[' + str(i) + ',' + str(v) + ',' + str(w) + ']'
                )

for i in Nodes:
    for j in Nodes:
        for v in Vehicles:
            for w in range(len(time_windows[int(i)])):   
                if i != j and i != 0 and j != 0:
                    con8[i,j,v,w] = m.addConstr(
                        service_start[j, v, w] >= service_start[i, v, w] + service_duration[int(i)] + distance[i, j] - BIGM * (1 - edge[i, j, v]), 'con8[' + str(i) + ',' + str(j) + ',' + str(v) + ',' + str(w) + ']'
                    )

# Waiting time
# for i in Nodes:
#     for v in Vehicles:
#         for w in range(len(time_windows[int(i)])):
#             a_iw, b_iw = time_windows[int(i)][w]
#             con9[i,v,w] = m.addConstr(service_start[i, v, w] + wait_time[i, v] <= a_iw * z[i, v, w], 'con9[' + str(i) + ',' + str(v) + ',' + str(w) + ']')

# Solve
m.update()
m.write('CVRPTW.lp')
m.Params.TimeLimit = 30600
m.optimize()

#Debugging
if m.status == GRB.INFEASIBLE:
    m.computeIIS() 

    for c in m.getConstrs():
        if c.IISConstr: 
            print('Unzulässige Constraint:', c.constrName)

    for v in m.getVars():
        if v.IISLB > 0 or v.IISUB > 0:
            print('Problem bei der Variable:', v.varName)


0
3390
0
3390
0
3390
0
3390
0
3390
0
3390
311
471
1011
1231
2700
3220
311
471
1011
1231
2700
3220
213
373
1053
1301
2654
2905
213
373
1053
1301
2654
2905
1167
1327
167
427
2677
2997
1167
1327
167
427
2677
2997
1261
1421
121
431
2791
2901
1261
1421
121
431
2791
2901
Set parameter TimeLimit to value 30600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10850H CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 172 rows, 110 columns and 510 nonzeros
Model fingerprint: 0xfbf84a30
Variable types: 30 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 162 rows and 101 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thre

In [3]:
# Extract the routes from the solution
routes = {v: [] for v in Vehicles}
if m.status == GRB.OPTIMAL:
    for v in Vehicles:
        for i in Nodes:
            for j in Nodes:
                if edge[i, j, v].X > 0.5:
                    routes[v].append((i, j))

    # Plot the routes
    plt.figure(figsize=(5, 4))
    plt.scatter(xcoord, ycoord, c='red')
    plt.scatter(xcoord[0], ycoord[0], c='blue', marker='s')  # depot

    for i, (x, y) in enumerate(zip(xcoord, ycoord)):
        plt.annotate(str(i), (x, y), textcoords="offset points", xytext=(0, 5), ha='center')

    colors = ['green', 'orange']  # Define colors for each vehicle
    for v in Vehicles:
        for (i, j) in routes[v]:
            plt.arrow(xcoord[i], ycoord[i], xcoord[j] - xcoord[i], ycoord[j] - ycoord[i], 
                      color=colors[v], head_width=1.5, length_includes_head=True, label=f'Vehicle {v}' if i == 0 else "")

    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Vehicle Routes')
    plt.legend()
    plt.show()
else:
    print("No optimal solution found.")


No optimal solution found.


In [8]:
# Create a dataframe to store the results for all vehicles
df_all = pd.DataFrame(columns=['Vehicle', 'Node', 'Service Start', 'Load'])

for v in Vehicles:
    print(f"Vehicle {v} service times and loads:")
    sorted_routes = sorted(routes[v], key=lambda x: service_start[x[0], v].X)
    load = 0
    for (i, j) in sorted_routes:
        load += q[i]
        print(f"  Node {i}: Service start at {service_start[i, v].X:.2f}, Load: {load}")
        new_row = pd.DataFrame({'Vehicle': [v], 'Node': [i], 'Service Start': [service_start[i, v].X], 'Load': [load]})
        if not new_row.isna().all(axis=None):
            df_all = pd.concat([df_all, new_row], ignore_index=True)

    print("\n")

df_all
# show the total distance travveled by all vehicles
total_distance = 0
for v in Vehicles:
    for (i, j) in routes[v]:
        total_distance += distance[i, j]
print(f"Total distance traveled by all vehicles: {total_distance:.2f}")

Vehicle 0 service times and loads:


KeyError: (0, 0)